In [78]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns

In [79]:
# Load data
kc2 = pd.read_csv(r'/content/kc2_csv.csv')
print('kc2 shape: ', kc2.shape)
kc2.head()

kc2 shape:  (522, 22)


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,problems
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,no
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,yes
2,415.0,59.0,50.0,51.0,1159.0,8411.31,0.01,103.53,81.24,870848.58,...,359,35,9,10,47.0,106.0,692.0,467.0,106.0,yes
3,230.0,33.0,10.0,16.0,575.0,3732.82,0.03,39.82,93.74,148644.06,...,174,15,34,5,23.0,67.0,343.0,232.0,65.0,yes
4,175.0,26.0,12.0,13.0,500.0,3123.96,0.03,29.48,105.96,92103.07,...,142,7,19,4,18.0,58.0,310.0,190.0,51.0,yes


In [80]:
kc2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 522 entries, 0 to 521
Data columns (total 22 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   loc               522 non-null    float64
 1   v(g)              522 non-null    float64
 2   ev(g)             522 non-null    float64
 3   iv(g)             522 non-null    float64
 4   n                 522 non-null    float64
 5   v                 522 non-null    float64
 6   l                 522 non-null    float64
 7   d                 522 non-null    float64
 8   i                 522 non-null    float64
 9   e                 522 non-null    float64
 10  b                 522 non-null    float64
 11  t                 522 non-null    float64
 12  lOCode            522 non-null    int64  
 13  lOComment         522 non-null    int64  
 14  lOBlank           522 non-null    int64  
 15  lOCodeAndComment  522 non-null    int64  
 16  uniq_Op           522 non-null    float64
 1

In [81]:
target = 'problems'
y = kc2[target]
x=kc2.drop(['problems'], axis=1)
x.shape

(522, 21)

after running " *from mlxtend.feature_selection import SequentialFeatureSelector as SFS* " this command you might get error this is because mlxtend is trying to import joblib from sklearn but it no longer exist there so fix this issue go to the file location (there will a file location link in the error message like this " /usr/local/lib/python3.7/dist-packages/mlxtend/feature_selection/sequential_feature_selector.py <module> " click on this location link)
then it will show you code written in that file then you just have to look for this line " from sklearn.externals.joblib import Parallel, delayed " and edit it to " from joblib import Parallel, delayed " and then save it by pressing ctrl+s. then run the code again. Again it will give error but this time the location link will be different and you have to again edit the file in same way as done priviouslly and save it and run the code again and this will solve the error.

In [82]:
from mlxtend.feature_selection import ExhaustiveFeatureSelector as EFS
from sklearn.svm import SVC
svc = SVC()


In [83]:
efs = EFS(svc, # can use any any learning method 
          min_features=1,# can set any limit from 1 to less then or equal to max_feature value
          max_features=4, # can set any value less then toatal number of features but greater then or equal to min_features
                          # higer the difference between the min and max value higher the time taken to select features
          scoring='accuracy',
          print_progress=True,
          cv=2
          ).fit(np.array(x[x.columns[0:5]].fillna(0)), y)
                                    # set value of columns like [0:x] where x is equal to value of max_features

Features: 30/30

In [84]:
efs.best_idx_

(3,)

In [85]:
x_reduced = efs.transform(x)
x_reduced.shape

(522, 1)

In [86]:
y.shape

(522,)

In [87]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_reduced, y, test_size=0.3, random_state=10)

In [88]:
print(x.shape)
print(y.shape)
print('-------------------')
print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(522, 21)
(522,)
-------------------
(365, 1)
(365,)
(157, 1)
(157,)


In [89]:
from sklearn.svm import SVC
from sklearn import metrics #for checking the model accuracy
svc = SVC()
svc.fit(x_train, y_train)
svc_pred=svc.predict(x_test)
from sklearn import metrics #for checking the model accuracy
acc_svm = metrics.accuracy_score(svc_pred,y_test)
print('The accuracy of the SVM is:',acc_svm)
print('Th confusion matrix of SVM:\n ',metrics.confusion_matrix(y_test, svc_pred))

The accuracy of the SVM is: 0.8407643312101911
Th confusion matrix of SVM:
  [[122   5]
 [ 20  10]]


In [90]:
print(metrics.confusion_matrix(y_test, svc_pred))
print(metrics.classification_report(y_test, svc_pred))

[[122   5]
 [ 20  10]]
              precision    recall  f1-score   support

          no       0.86      0.96      0.91       127
         yes       0.67      0.33      0.44        30

    accuracy                           0.84       157
   macro avg       0.76      0.65      0.68       157
weighted avg       0.82      0.84      0.82       157



In [91]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=2) #put 2 neighbors because we have 2 labels.
knn.fit(x_train,y_train)
knn_pred = knn.predict(x_test)
acc_knn = metrics.accuracy_score(knn_pred,y_test)
print('The accuracy of the KNN is', acc_knn)
print('The confusion matrix of KNN:\n ',metrics.confusion_matrix(y_test, knn_pred))
print(metrics.confusion_matrix(y_test, knn_pred))
print(metrics.classification_report(y_test, knn_pred))

The accuracy of the KNN is 0.7898089171974523
The confusion matrix of KNN:
  [[121   6]
 [ 27   3]]
[[121   6]
 [ 27   3]]
              precision    recall  f1-score   support

          no       0.82      0.95      0.88       127
         yes       0.33      0.10      0.15        30

    accuracy                           0.79       157
   macro avg       0.58      0.53      0.52       157
weighted avg       0.73      0.79      0.74       157



In [92]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)
y_predict = model.predict(x_test)
model_score = model.score(x_test, y_test)
print(model_score)
print(metrics.confusion_matrix(y_test, y_predict))
print(metrics.classification_report(y_test, y_predict))

0.821656050955414
[[121   6]
 [ 22   8]]
              precision    recall  f1-score   support

          no       0.85      0.95      0.90       127
         yes       0.57      0.27      0.36        30

    accuracy                           0.82       157
   macro avg       0.71      0.61      0.63       157
weighted avg       0.79      0.82      0.79       157



In [93]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()
model.fit(x_train, y_train)
model.score(x_train, y_train) 

0.8356164383561644

In [94]:
test_pred = model.predict(x_test)
print(metrics.classification_report(y_test, test_pred))
print(metrics.confusion_matrix(y_test, test_pred))

              precision    recall  f1-score   support

          no       0.86      0.96      0.91       127
         yes       0.67      0.33      0.44        30

    accuracy                           0.84       157
   macro avg       0.76      0.65      0.68       157
weighted avg       0.82      0.84      0.82       157

[[122   5]
 [ 20  10]]
